In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("theappname").getOrCreate()
df = spark.read.csv('/Code/spark/LAB4/data/20*.csv', mode="DROPMALFORMED", inferSchema=True, header = True)
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    1|         1|        1|   1232|      1225|   1341|      1340|           WN|     2891

In [148]:
@udf(returnType=TimestampType())
def convertToDateTime( year, month, day, arrTime ): 
    print(arrTime)
    return datetime.strptime(str(year).zfill(4)+str(month).zfill(2)+str(day).zfill(2)+' '+arrTime+"00", '%Y%m%d %H%M%S')


def convertToDateTime2( year, month, day, arrTime ): 
    return datetime.strptime(str(year).zfill(4)+str(month).zfill(2)+str(day).zfill(2)+' '+arrTime+"00", '%Y%m%d %H%M%S')

print(convertToDateTime2(2008, 1, 2, "1200"))

2008-01-02 12:00:00


In [3]:
from pyspark.sql.functions import monotonically_increasing_id, lag, length

@udf(returnType=IntegerType())
def nbrOfPreviousLateLadning(s):
    return s.lag(s.ArrTime, 1, 0) > 15 if 1 else 0 +  (s.lag(s.ArrTime, 2, 0) > 15 if 1 else 0)

In [52]:
@udf(returnType=IntegerType())
def getNumberOfLateFlightsLastHour(s, farrTime, dest):
    return s.where((s.Year == farrTime.year) & (s.Month == farrTime.month) & (s.DayofMonth == farrTime.day) & (s.Dest == "FLL") & (s.ArrDelay > 15) & (s.FullArrTimeStamp < farrTime) & (s.FullArrTimeStamp > (farrTime- timedelta(hours=4))) ).count()

In [143]:
 df.select('ArrTime')\
    .filter(length(df.ArrTime) == 4 ).show()
#    .withColumn('Arrtimelengh', length(df.ArrTime)).show()
#.filter(length(df.ArrTime) != 2 ).distinct().sort('ArrTime').take(31)


+-------+
|ArrTime|
+-------+
|   1341|
|   2043|
|   2334|
|   1356|
|   1553|
|   2217|
|   1223|
|   1819|
|   1603|
|   1119|
|   2332|
|   2245|
|   1048|
|   1405|
|   1843|
|   1158|
|   1839|
|   2016|
|   2332|
|   2131|
+-------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import monotonically_increasing_id, lag, length

#convertToDateTime_udf  = udf(lambda year, month, day, arrTime: convertToDateTime(year, month, day, arrTime)  , TimestampType())
from pyspark.sql.types import IntegerType
selection = df \
    .where(df.CancellationCode.isNull()) \
    .filter(length(df.ArrTime) == 4 ) \
    .filter(df.Diverted == 0)\
    .sort(asc('Year'), 'Month', 'DayofMonth', 'ArrTime')\
    .withColumn("ArrDelay", df.ArrDelay.cast(IntegerType()))\
    .withColumn("DepDelay", df["DepDelay"].cast(IntegerType()))\
    .withColumn("CarrierDelay", df["CarrierDelay"].cast(IntegerType()))\
    .withColumn("WeatherDelay", df["WeatherDelay"].cast(IntegerType()))\
    .withColumn("NASDelay", df["NASDelay"].cast(IntegerType()))\
    .withColumn("SecurityDelay", df["SecurityDelay"].cast(IntegerType()))\
    .withColumn("LateAircraftDelay", df["LateAircraftDelay"].cast(IntegerType()))\
    .fillna(0, subset=['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay' ])
#    .withColumn('id', monotonically_increasing_id())
#    .withColumn("FullArrTimeStamp", convertToDateTime(df.Year, df.Month, df.DayofMonth, df.ArrTime)).collect()
    
#    .withColumn("NbrOfPreviousLateFlights", nbrOfPreviousLateLadning(col["ArrTime"]) )

#selection.sort(col.FullArrTimeStamp).show()

In [109]:
selection.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

In [32]:
from pyspark.sql.functions import lag
from pyspark.sql import Window
w = Window.orderBy("Year", "Month", "DayofMonth", "ArrTime")

sorted = selection \
    .withColumn("NbrOfPreviousLateFlights",  (lag(selection['ArrDelay'], 1,0).over(w)  > 15 if 1 else 0).cast(IntegerType()) + (lag(selection['ArrDelay'], 2,0).over(w)  > 15 if 1 else 0).cast(IntegerType())+ (lag(selection['ArrDelay'], 3,0).over(w)  > 15 if 1 else 0).cast(IntegerType()))\
    .withColumn("IsLate", (selection['ArrDelay'] > 15).cast(IntegerType())  )\
    .show()
#    selection.withColumn("NbrOfPreviousLateFlifhts", getNumberOfLateFlightsLastHour(selection, convertToDateTime(selection.Year, selection.Month, selection.DayofMonth, selection.ArrTime), selection.Dest) ).show()



+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------------------+------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|NbrOfPreviousLateFlights|IsLate|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------------------+------

In [61]:


#def getNumberOfLateFlightsLastHour(s, farrTime, dest):

#    return s.where((s.Year == farrTime.year) & (s.Month == farrTime.month) & (s.DayofMonth == farrTime.day) & (s.Dest == "FLL") & (s.ArrDelay > 15) & (s.FullArrTimeStamp < farrTime) & (s.FullArrTimeStamp > (farrTime- timedelta(hours=4))) )

yyy = convertToDateTime2(2008, 1, 2, "1200")
getNumberOfLateFlightsLastHour(selection, yyy, "FLL").count()
    

11

In [ ]:
selection.map()

In [110]:
selection.createOrReplaceTempView("table1")



In [113]:
spark.sql("select Year, Month, DayofMonth, FlightNum, Dest, (select count(*) from table1 where year == t.year and month == t.month and DayofMonth == t.DayofMonth and Dest == t.Dest and ArrDelay > 15 ) from table1 t").show()

+----+-----+----------+---------+----+---------------------------------------------+
|Year|Month|DayofMonth|FlightNum|Dest|scalarsubquery(year, month, DayofMonth, Dest)|
+----+-----+----------+---------+----+---------------------------------------------+
|2007|    1|         2|     5316| HTS|                                            0|
|2007|    1|         2|     5061| HTS|                                            0|
|2007|    1|         2|      156| SAT|                                           34|
|2007|    1|         2|     1350| SAT|                                           34|
|2007|    1|         2|     2472| SAT|                                           34|
|2007|    1|         2|     2123| SAT|                                           34|
|2007|    1|         2|     1635| SAT|                                           34|
|2007|    1|         2|     2613| SAT|                                           34|
|2007|    1|         2|      515| SAT|                           

In [84]:
import pandas as pd 

selection.fillna(0, inplace=True)
selection["late"] = selection["ArrDelay"] > 15
selection.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,late
0,2007,8,1,3,1611,1610,1801,1822,XE,3063,...,29,0,0,0,0,0,0,0,0,False
1,2008,1,3,4,859,835,1101,1048,EV,4197,...,12,0,0,0,0,0,0,0,0,False
2,2008,10,16,4,1645,1645,1840,1842,9E,2013,...,12,0,0,0,0,0,0,0,0,False
3,2008,8,20,3,1410,1410,1547,1515,AA,1282,...,16,0,0,0,0,0,32,0,0,True
4,2007,2,1,4,1725,1710,1945,1950,WN,1350,...,8,0,0,0,0,0,0,0,0,False


In [89]:
import numpy as np

selection.where(selection["late"] == True).dropna().count() / selection.count()


Year                 0.22278
Month                0.22278
DayofMonth           0.22278
DayOfWeek            0.22278
DepTime              0.22278
CRSDepTime           0.22278
ArrTime              0.22278
CRSArrTime           0.22278
UniqueCarrier        0.22278
FlightNum            0.22278
TailNum              0.22278
ActualElapsedTime    0.22278
CRSElapsedTime       0.22278
AirTime              0.22278
ArrDelay             0.22278
DepDelay             0.22278
Origin               0.22278
Dest                 0.22278
Distance             0.22278
TaxiIn               0.22278
TaxiOut              0.22278
Cancelled            0.22278
CancellationCode     0.22278
Diverted             0.22278
CarrierDelay         0.22278
WeatherDelay         0.22278
NASDelay             0.22278
SecurityDelay        0.22278
LateAircraftDelay    0.22278
late                 0.22278
dtype: float64